<a href="https://colab.research.google.com/github/leeyunche/hello-colab/blob/main/Mask_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json

# 利用 requests 對 API 來源發送一個請求
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
s = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(s)

print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# 計算各地區藥局數量
med_count = {}

# 填入欄位名稱
for s in data["features"]:
    county = s["properties"]["address"][:3]
    med_count[county] = med_count.get(county, 0) + 1  # 若county在字典中，返回縣市出現的次數再加1，若無則返回0

print(med_count)

{'臺北市': 340, '高雄市': 424, '臺中市': 436, '臺南市': 280, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 507, '桃園市': 271, '新竹縣': 48, '宜蘭縣': 76, '苗栗縣': 57, '彰化縣': 181, '南投縣': 67, '雲林縣': 131, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 47, '臺東縣': 23, '金門縣': 6, '連江縣': 1}


In [ ]:
# 計算出每個地區的成人剩餘口罩數量，將結果從大到小排列

mask_count = {}

# 填入欄位名稱
for s in data["features"]:
    county = s["properties"]["address"][:3]
    mask_count[county] = mask_count.get(county, 0) + s["properties"]["mask_adult"]

# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1], reverse = True))

print(mask_count)

{'新北市': 677660, '臺中市': 556440, '高雄市': 428940, '臺南市': 408400, '桃園市': 345600, '臺北市': 323660, '彰化縣': 219400, '雲林縣': 199470, '屏東縣': 184250, '苗栗縣': 134180, '嘉義縣': 113180, '嘉義市': 107840, '宜蘭縣': 106900, '南投縣': 91240, '基隆市': 80100, '花蓮縣': 57330, '新竹縣': 53760, '新竹市': 45630, '臺東縣': 28510, '金門縣': 15510, '澎湖縣': 12600, '連江縣': 7930}
